In [1]:
import os
import sys
sys.path.append('..')
from data_center.tej_handler import TEJHandler
from monitor.tw_monitor import TWMarketMonitor

from pymongo import MongoClient
import tejapi

import pandas as pd
import numpy as np
from datetime import date
from dateutil.relativedelta import relativedelta
# TEJHandler()

In [2]:
pdata = TWMarketMonitor().get_data_from_mongo('PSTAGE', 'financial_report', limit = 10000)
pdata

KeyError: 'database'

In [ ]:
mongo = MongoClient(TWMarketMonitor().client_url)
db = mongo['PSTAGE']
collection = db['margin_trading']
df = pd.DataFrame(list(collection.find().sort('財務資料日', -1).limit(1000000)))
df

,_id,證券碼,財務資料日,季別,合併(Y/N),單季(Q)/單半年(H),月份,幣別,財報發布日,每股盈餘,...,營業利益,營業外收入及支出,所得稅費用,合併總損益,歸屬母公司淨利（損）,折舊－CFO,攤提－CFO,來自營運之現金流量,投資活動之現金流量,籌資活動之現金流量
0,660cd7f1673ad9148a129469,1101,2023-12-01,4,Y,Q,12,NTD,2024-02-28,0.25,...,3433504.0,383504.0,1317094.0,2499914,1801908,2082429.0,282945.0,13796962.0,-11346930.0,-5820763.0
1,660cd7f1673ad9148a1294f4,1103,2023-12-01,4,Y,Q,12,NTD,2024-02-29,-0.08,...,-20970.0,-46157.0,-15624.0,-51503,-54360,110085.0,671.0,84138.0,-214439.0,-2268.0
2,660cd7f1673ad9148a1297a7,1216,2023-12-01,4,Y,Q,12,NTD,2024-03-07,0.27,...,4521577.0,1969696.0,2915000.0,3576273,1562387,10352526.0,343745.0,16412096.0,-11161173.0,-6706368.0
3,660cd7f1673ad9148a129a15,1232,2023-12-01,4,Y,Q,12,NTD,2024-02-23,2.06,...,428744.0,-6644.0,80728.0,341372,329943,66624.0,206.0,342154.0,-99204.0,-86820.0
4,660cd7f1673ad9148a129b2a,1236,2023-12-01,4,Y,Q,12,NTD,2024-02-29,0.02,...,-7498.0,4784.0,-4950.0,2236,2236,37543.0,9789.0,65426.0,-60536.0,-22957.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106048,660cd7f1673ad9148a1431cd,8712,2005-06-01,2,N,H,6,NTD,2005-08-30,-2.70,...,-147867.0,-2581873.0,0.0,-2729740,-2729740,10228.0,3389.0,-31897.0,323676.0,-287406.0
106049,660cd7f1673ad9148a1431ce,8725,2005-06-01,2,N,H,6,NTD,2006-08-30,-3.65,...,-7160.0,-90791.0,0.0,-423249,-423249,158.0,0.0,18339.0,-1529.0,-37528.0
106050,660cd7f1673ad9148a1431d3,8913,2005-06-01,2,Y,H,6,NTD,2006-08-29,-0.03,...,-15436.0,28417.0,1944.0,11037,11037,60924.0,36.0,128306.0,133327.0,-271975.0
106051,660cd7f1673ad9148a143208,8925,2005-06-01,2,N,H,6,NTD,2006-08-29,0.19,...,41628.0,-14578.0,5930.0,21120,21120,21576.0,5024.0,-21634.0,-28630.0,-51783.0


In [ ]:
data_code = "TWN/EWIFINQ"
pdata = TWMarketMonitor().pdata_pipline(df, data_code)
pdata['每股盈餘']['2013':].sort_index(ascending=False)

symbol,1101,1102,1103,1104,1107,1108,1109,1110,1113,1201,...,9944,9945,9946,9949,9950,9951,9955,9958,9960,9962
datetime,,,,,,,,,,,,,,,,,,,,,
2023-12-01,0.25,NaN,-0.08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.23,NaN,NaN,NaN,NaN,1.95,NaN,0.14
2023-09-01,0.17,0.82,0.18,1.42,NaN,0.47,0.34,0.10,NaN,0.12,...,-0.61,2.10,-0.06,-0.20,0.48,1.52,-0.20,1.32,0.97,0.32
2023-06-01,0.45,1.26,0.00,0.62,NaN,0.37,0.60,0.12,NaN,0.07,...,-0.60,1.52,0.08,-0.20,5.58,1.44,-0.02,0.99,0.37,0.33
2023-03-01,0.20,0.81,0.03,0.42,NaN,0.31,0.42,0.06,NaN,0.25,...,2.92,0.10,0.01,-0.23,0.15,1.35,-0.31,0.20,0.59,0.48
2022-12-01,0.36,0.18,0.05,0.93,NaN,0.68,0.34,0.19,NaN,-0.17,...,1.63,-1.66,0.13,0.14,-0.12,1.82,-0.27,-0.61,0.86,0.58
2022-09-01,0.22,1.33,0.41,0.63,NaN,0.33,0.63,0.12,NaN,0.28,...,-0.25,1.47,0.20,-0.24,0.10,1.78,-0.29,0.68,1.38,0.67
2022-06-01,-0.02,1.35,-0.62,0.51,NaN,0.29,-0.06,-0.10,NaN,0.18,...,0.21,2.87,0.05,-0.47,-0.91,0.74,-0.24,0.05,0.81,0.70
2022-03-01,0.20,0.76,-0.12,1.06,NaN,0.14,0.26,0.05,NaN,0.19,...,0.54,1.91,0.05,-0.22,-0.58,1.16,-0.15,1.47,0.57,0.41
2021-12-01,0.90,1.25,0.09,0.55,NaN,0.23,0.91,0.03,NaN,0.24,...,4.26,1.02,0.22,0.05,-0.74,1.18,-0.32,0.80,0.64,0.62


In [ ]:
pdata['財報發布日']['2013':].sort_index(ascending=False)

symbol,1101,1102,1103,1104,1107,1108,1109,1110,1113,1201,...,9944,9945,9946,9949,9950,9951,9955,9958,9960,9962
datetime,,,,,,,,,,,,,,,,,,,,,
2023-12-01,2024-02-28,NaT,2024-02-29,NaT,NaT,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,2024-02-23,NaT,NaT,NaT,NaT,2024-03-08,NaT,2024-03-12
2023-09-01,2023-11-10,2023-11-13,2023-11-10,2023-11-13,NaT,2023-11-13,2023-11-09,2023-11-10,NaT,2023-11-08,...,2023-11-13,2023-11-13,2023-11-13,2023-11-09,2023-11-14,2023-11-06,2023-11-09,2023-11-13,2023-11-10,2023-11-13
2023-06-01,2023-08-10,2023-08-11,2023-08-11,2023-08-11,NaT,2023-08-14,2023-08-10,2023-08-11,NaT,2023-08-04,...,2023-08-11,2023-08-11,2023-08-14,2023-08-09,2023-08-14,2023-08-08,2023-08-09,2023-08-11,2023-08-10,2023-08-09
2023-03-01,2023-05-11,2023-05-12,2023-05-11,2023-05-15,NaT,2023-05-12,2023-05-12,2023-05-11,NaT,2023-05-11,...,2023-05-12,2023-05-12,2023-05-11,2023-05-12,2023-05-15,2023-05-09,2023-05-15,2023-05-15,2023-05-11,2023-05-05
2022-12-01,2023-02-24,2023-03-16,2023-03-14,2023-03-17,NaT,2023-03-31,2023-03-14,2023-03-16,NaT,2023-03-15,...,2023-03-30,2023-03-14,2023-02-24,2023-03-30,2023-03-29,2023-03-23,2023-03-23,2023-03-31,2023-03-29,2023-03-23
2022-09-01,2022-11-10,2022-11-11,2022-11-08,2022-11-14,NaT,2022-11-10,2022-11-11,2022-11-11,NaT,2022-11-04,...,2022-11-11,2022-11-11,2022-11-09,2022-11-11,2022-11-12,2022-11-08,2022-11-10,2022-11-11,2022-11-11,2022-11-10
2022-06-01,2022-08-12,2022-08-12,2022-08-11,2022-08-12,NaT,2022-08-12,2022-08-11,2022-08-11,NaT,2022-08-05,...,2022-08-12,2022-08-12,2022-08-04,2022-08-10,2022-08-12,2022-08-09,2022-08-02,2022-08-15,2022-08-10,2022-08-10
2022-03-01,2022-05-10,2022-05-12,2022-05-06,2022-05-13,NaT,2022-05-13,2022-05-12,2022-05-12,NaT,2022-05-06,...,2022-05-13,2022-05-13,2022-05-09,2022-05-11,2022-05-13,2022-05-09,2022-05-04,2022-05-13,2022-05-12,2022-05-10
2021-12-01,2022-02-25,2022-03-31,2022-03-22,2022-03-30,NaT,2022-03-31,2022-03-30,2022-03-18,NaT,2022-03-23,...,2022-03-30,2022-03-15,2022-03-30,2022-03-25,2022-03-30,2022-03-24,2022-03-29,2022-03-31,2022-03-29,2022-03-16


In [ ]:
pdata.keys().to_list()

AttributeError: 'dict_keys' object has no attribute 'to_list'

In [ ]:
get_historical_returns = TWMarketMonitor().get_market_returns(review_month = 1)